## Parameter sets
Change, create and use user-defined parameter sets

In [1]:
!pip install pybamm
import pybamm as pb

c:\ProgramData\anaconda3\Lib\site-packages\paramiko\transport.py:219: CryptographyDeprecationWarning: Blowfish has been deprecated and will be removed in a future release
  "class": algorithms.Blowfish,


In [2]:
# parameter values are held in a dictionary, which can be looked up via:
parameter_values = pb.ParameterValues("Marquis2019")
print(parameter_values["Upper voltage cut-off [V]"])
print(parameter_values["Lower voltage cut-off [V]"])

# initialise the model
model = pb.lithium_ion.DFN()

4.1
3.105


In [3]:
experiment3 = pb.Experiment(
    # Initialize
    [
        "Hold at 4.2 V until C/100",
        "Rest for 4 hours",
    ]
    # Capacity check
    + [(
        "Discharge at C/10 until 2.5 V",
        "Charge at C/10 until 4.2 V",
        "Hold at 4.2 V until C/100"
    )]
    # Ageing cycles
    + [(
        "Discharge at 1C until 2.5 V",
        "Charge at 0.3C until 4.2 V",
        "Hold at 4.2 V until C/100",
    )] * 10
    # Capacity check
    + [(
        "Discharge at C/10 until 2.5 V",
        "Charge at C/10 until 4.2 V",
        "Hold at 4.2 V until C/100"
    )]
)

# attempt simulation using this set
simulation3 = pb.Simulation(model, experiment=experiment3)
solution = simulation3.solve([0, 3600])
simulation3.plot()

c:\ProgramData\anaconda3\Lib\site-packages\pybamm\simulation.py:120: UserWarning: The default solver changed to IDAKLUSolver after the v25.4.0. release. You can swap back to the previous default by using `pybamm.CasadiSolver()` instead.
  self._solver = solver or self._model.default_solver
2025-07-15 10:45:55.487 - [WARNING] simulation.solve(551): Ignoring t_eval as solution times are specified by the experiment


interactive(children=(FloatSlider(value=0.0, description='t', max=121.18261591527377, step=1.2118261591527377)…

In [9]:
# attempt simulation using "OKane2022" parameter set
parameter_values = pb.ParameterValues("OKane2022") 
print(parameter_values)

simulation3 = pb.Simulation(
    model,
    experiment=experiment3,
    parameter_values=parameter_values
)

sol = simulation3.solve([0,1800])
simulation3.plot()

{'Ambient temperature [K]': 298.15,
 'Boltzmann constant [J.K-1]': 1.380649e-23,
 'Bulk solvent concentration [mol.m-3]': 2636.0,
 'Cation transference number': 0.2594,
 'Cell cooling surface area [m2]': 0.00531,
 'Cell thermal expansion coefficient [m.K-1]': 1.1e-06,
 'Cell volume [m3]': 2.42e-05,
 'Contact resistance [Ohm]': 0,
 'Current function [A]': 5.0,
 'Dead lithium decay constant [s-1]': 1e-06,
 'Dead lithium decay rate [s-1]': <function SEI_limited_dead_lithium_OKane2022 at 0x000002672C6F0EA0>,
 'EC diffusivity [m2.s-1]': 2e-18,
 'EC initial concentration in electrolyte [mol.m-3]': 4541.0,
 'Electrode height [m]': 0.065,
 'Electrode width [m]': 1.58,
 'Electrolyte conductivity [S.m-1]': <function electrolyte_conductivity_Nyman2008_arrhenius at 0x000002672C6F1300>,
 'Electrolyte diffusivity [m2.s-1]': <function electrolyte_diffusivity_Nyman2008_arrhenius at 0x000002672C6F1A80>,
 'Electron charge [C]': 1.602176634e-19,
 'Exchange-current density for plating [A.m-2]': <function 

2025-07-15 10:48:34.182 - [WARNING] simulation.solve(551): Ignoring t_eval as solution times are specified by the experiment


interactive(children=(FloatSlider(value=0.0, description='t', max=95.5281279779589, step=0.955281279779589), O…

In [21]:
# change/update the parameter set for a more significant degradation
parameter_values = pb.ParameterValues("OKane2022")
print(parameter_values)

parameter_values.update({
    "SEI solvent diffusivity [m2.s-1]": 1.25e-20,
    "Lithium plating kinetic rate constant [m.s-1]": 1e-8,
    "Dead lithium decay constant [s-1]": 4e-6,
    "Negative electrode cracking rate": 1.95e-18,
    "Negative electrode LAM constant proportional term [s-1]": 5.5556e-6,
    "Positive electrode LAM constant proportional term [s-1]": 5.5556e-6,
})

simulation3 = pb.Simulation(
    model,
    experiment=experiment3,
    parameter_values=parameter_values
)
sol2 = simulation3.solve([0, 1800])
sol2.plot()

{'Ambient temperature [K]': 298.15,
 'Boltzmann constant [J.K-1]': 1.380649e-23,
 'Bulk solvent concentration [mol.m-3]': 2636.0,
 'Cation transference number': 0.2594,
 'Cell cooling surface area [m2]': 0.00531,
 'Cell thermal expansion coefficient [m.K-1]': 1.1e-06,
 'Cell volume [m3]': 2.42e-05,
 'Contact resistance [Ohm]': 0,
 'Current function [A]': 5.0,
 'Dead lithium decay constant [s-1]': 1e-06,
 'Dead lithium decay rate [s-1]': <function SEI_limited_dead_lithium_OKane2022 at 0x000002672C6F0EA0>,
 'EC diffusivity [m2.s-1]': 2e-18,
 'EC initial concentration in electrolyte [mol.m-3]': 4541.0,
 'Electrode height [m]': 0.065,
 'Electrode width [m]': 1.58,
 'Electrolyte conductivity [S.m-1]': <function electrolyte_conductivity_Nyman2008_arrhenius at 0x000002672C6F1300>,
 'Electrolyte diffusivity [m2.s-1]': <function electrolyte_diffusivity_Nyman2008_arrhenius at 0x000002672C6F1A80>,
 'Electron charge [C]': 1.602176634e-19,
 'Exchange-current density for plating [A.m-2]': <function 

2025-07-15 10:55:48.108 - [WARNING] simulation.solve(551): Ignoring t_eval as solution times are specified by the experiment


interactive(children=(FloatSlider(value=0.0, description='t', max=95.5281279779589, step=0.955281279779589), O…

You can create and import your own parameter sets:

```python
from your_parameter_set import get_parameter_values
parameter_values = get_parameter_values()
```

## Input parameters

This is useful for wanting to see how one or a small subset of parameter affects the solution. Instead of recreating a "simulation" object, use an "input parameter":

In [22]:
parameter_values = pb.ParameterValues("Marquis2019")
parameter_values.update({
    "Current function [A]": "[input]",
})

# allows input of a parameter when the model gets solved
simulation = pb.Simulation(
    model,
    parameter_values=parameter_values
)
solution = simulation.solve([0, 3600], inputs={"Current function [A]": 2})
simulation.plot()

interactive(children=(FloatSlider(value=0.0, description='t', max=1173.9686106231375, step=11.739686106231375)…